<div style="text-align: center; color: black;">
    <h1>🛢️ Export Synthetic HR Dataset to MySQL Database</h1>
</div>

### 🛡️ Step 1: Retrieve MySQL Password Securely
To maintain security and avoid hardcoding sensitive credentials, we fetch the MySQL password from a system-level environment variable. This helps keep your notebook portable and safe for version control or collaboration. <br>
If the password isn't set, an error message will be displayed prompting the user to verify their environment setup.

In [1]:
# Import the OS module to access environment variables
import os
# Retrieve the MySQL password stored as an environment variable
mysql_password = os.getenv("MySQL_Password")
# Check whether the password was successfully retrieved
if mysql_password is None:
    # Display a warning if the environment variable is missing
    print("Password not found in environment variables!")
else:
    # Confirm successful retrieval (without printing the password itself)
    print("Password retrieved successfully!")

Password retrieved successfully!


### 🧰 Step 2: Import Libraries for Database Integration and Data Handling

We import key Python libraries required for this data export workflow:
- `pandas` and `numpy` for structured data manipulation
- `mysql.connector` to establish a direct connection to the MySQL database

These form the backbone of the ETL process as we move data from CSV files into relational tables.

In [2]:
# Import core libraries for data manipulation and database connectivity
import numpy as np
import pandas as pd
import mysql.connector

### 📤 Step 3: Export Attrition Records to MySQL

In this step, we load the generated `attrition.csv` file and insert its contents into the `attrition` table within the `hr_synthetic` MySQL database. This dataset captures employee exit events, including exit date, reason, and satisfaction level.

Before export, we:
- Convert the exit date column to datetime format
- Connect securely to the MySQL database using `mysql.connector`
- Loop through each record and insert it into the `attrition` table

In [3]:
# Load the attrition dataset from the synthetic data directory
df_attrition = pd.read_csv("./hr_synthetic_data/attrition.csv")

# Convert 'exit_date' to datetime format for proper insertion into MySQL
df_attrition["exit_date"] = pd.to_datetime(df_attrition["exit_date"])

# Establish connection to the MySQL database using stored credentials
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Loop through each row and insert into the attrition table
for _, row in df_attrition.iterrows():
    cursor.execute("""
        INSERT INTO attrition (employee_id, exit_date, reason, voluntary_exit, satisfaction_rating)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

### 🗂️ Step 4: Export Department Reference Data to MySQL

This step loads the static `departments.csv` reference file and inserts department records into the `departments` table. Each record includes:
- A unique `department_id`
- The corresponding `department_name`

This table acts as a lookup for job roles and employee affiliations throughout the HR dataset.

In [4]:
# Load the department reference dataset from CSV
df_dept = pd.read_csv("./hr_synthetic_data/departments.csv")

# Establish connection to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Loop through each row and insert into the departments table
for _, row in df_dept.iterrows():
    cursor.execute("""
        INSERT INTO departments (department_id, department_name)
        VALUES (%s, %s)
    """, tuple(row))

# Commit transaction and close connection
conn.commit()
cursor.close()
conn.close()

### 👥 Step 5: Export Employee Records to MySQL (Chunked Insertion)

Given the large size of the `employees.csv` file, we process and insert data in manageable chunks (e.g. 5000 rows at a time). This improves memory efficiency and database performance.

Each employee record includes:
- Personal details (name, age, gender, marital status, etc.)
- Employment metadata (job role, department, hire date, salary)
- All dates are converted to proper datetime format before insertion

In [6]:
# Define the number of rows to process at a time
chunk_size = 5000

# Establish connection to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Read the employee CSV in chunks for memory-efficient processing
for chunk in pd.read_csv("./hr_synthetic_data/employees.csv", chunksize=chunk_size):
    
    # Convert 'hire_date' to datetime for proper SQL formatting
    chunk["hire_date"] = pd.to_datetime(chunk["hire_date"])
    
    # Insert each record into the employees table
    for _, row in chunk.iterrows():
        cursor.execute("""
            INSERT INTO employees (
                employee_id, name, age, gender, marital_status,
                ethnicity, country, job_role_id, department_id, hire_date, salary
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, tuple(row))

    # Commit after each chunk is inserted
    conn.commit()

# Close cursor and connection to release resources
cursor.close()
conn.close()

### 📋 Step 6: Export Hiring Source Data to MySQL

This final export step loads the `hiring_sources.csv` file and inserts records into the `hiring_sources` table of the MySQL database. The dataset captures:

- The source through which each employee was recruited
- Referral information (if applicable)

Before export, we sanitize the `referred_by` field by replacing blank and invalid entries with `None` to ensure compatibility with MySQL's NULL values.

In [7]:
# Load the hiring sources dataset from the synthetic data directory
df_hiring = pd.read_csv("./hr_synthetic_data/hiring_sources.csv")

# Sanitize the 'referred_by' column by replacing blanks and invalid entries with None
df_hiring["referred_by"] = df_hiring["referred_by"].replace(["", "nan", np.nan, pd.NA], None)

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Insert each row into the hiring_sources table
for _, row in df_hiring.iterrows():
    cursor.execute("""
        INSERT INTO hiring_sources (employee_id, hiring_source, referred_by)
        VALUES (%s, %s, %s)
    """, tuple(row))

# Finalize transaction and release database resources
conn.commit()
cursor.close()
conn.close()

### 👔 Step 7: Export Job Roles to MySQL

This step loads the structured `job_roles.csv` file and inserts records into the `job_roles` table. Each job role is mapped to its parent department using a `department_id`, supporting organizational hierarchy and analytical joins.

These role definitions are essential for tracking employee function and departmental composition.

In [5]:
# Load the job roles dataset from CSV
df_roles = pd.read_csv("./hr_synthetic_data/job_roles.csv")

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Insert each job role record into the job_roles table
for _, row in df_roles.iterrows():
    cursor.execute("""
        INSERT INTO job_roles (job_role_id, job_role_name, department_id)
        VALUES (%s, %s, %s)
    """, tuple(row))

# Finalize transaction and release database resources
conn.commit()
cursor.close()
conn.close()

### 📊 Step 8: Export Performance Review Data to MySQL (Chunked Insertion)

This section loads and inserts synthetic performance review records into the `performance_reviews` table. Each entry includes:

- Annual review date
- Performance score (1–5 scale)
- Bonus eligibility indicator

Chunked processing ensures the notebook remains memory-efficient when handling large volumes of employee evaluations.

In [8]:
# Define the number of records to process per chunk
chunk_size = 5000

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Read the performance review CSV in chunks
for chunk in pd.read_csv("./hr_synthetic_data/performance_reviews.csv", chunksize=chunk_size):
    
    # Convert 'review_date' to datetime for correct SQL formatting
    chunk["review_date"] = pd.to_datetime(chunk["review_date"])
    
    # Insert each row into the performance_reviews table
    for _, row in chunk.iterrows():
        cursor.execute("""
            INSERT INTO performance_reviews (
                employee_id, review_date, performance_score, bonus_eligible
            )
            VALUES (%s, %s, %s, %s)
        """, tuple(row))
    
    # Commit after processing each chunk
    conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

### 🎓 Step 9: Export Promotion Records to MySQL

This step loads the `promotions.csv` dataset and inserts promotion events into the `promotions` table of the MySQL database. Each record includes:

- `employee_id`: the individual who was promoted
- `promotion_date`: when the promotion occurred
- `new_job_role_id`: the role assigned after promotion

Promotion dates are converted to datetime format before insertion to ensure accurate SQL representation.

In [9]:
# Load promotion records from CSV
df_promotions = pd.read_csv("./hr_synthetic_data/promotions.csv")

# Convert 'promotion_date' to datetime for proper SQL formatting
df_promotions["promotion_date"] = pd.to_datetime(df_promotions["promotion_date"])

# Establish connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Insert each promotion record into the promotions table
for _, row in df_promotions.iterrows():
    cursor.execute("""
        INSERT INTO promotions (employee_id, promotion_date, new_job_role_id)
        VALUES (%s, %s, %s)
    """, tuple(row))

# Commit transaction and release resources
conn.commit()
cursor.close()
conn.close()

### 💰 Step 10: Export Salary History to MySQL (Chunked Insertion)

The final export step inserts employee salary progression data from `salaries.csv` into the `salaries` table. Each record includes:

- Employee ID
- Salary effective date (converted to datetime format)
- Salary amount

To ensure performance and scalability, the dataset is loaded and inserted in chunks (e.g. 5000 records at a time), making it suitable for large datasets and production-level workflows.

In [10]:
# Define the number of records to load per chunk
chunk_size = 5000

# Establish a secure connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="hr_synthetic"
)
cursor = conn.cursor()

# Load the salaries CSV in chunks and insert into the database
for chunk in pd.read_csv("./hr_synthetic_data/salaries.csv", chunksize=chunk_size):
    
    # Convert 'effective_date' column to datetime format
    chunk["effective_date"] = pd.to_datetime(chunk["effective_date"])
    
    # Insert each row into the salaries table
    for _, row in chunk.iterrows():
        cursor.execute("""
            INSERT INTO salaries (
                employee_id, effective_date, salary_amount
            )
            VALUES (%s, %s, %s)
        """, tuple(row))
    
    # Commit after each chunk is inserted
    conn.commit()

# Close database connection
cursor.close()
conn.close()

### ✅ Final Summary

In this notebook, we successfully exported a comprehensive synthetic HR dataset from structured CSV files into a normalized MySQL database. Each table reflects real-world employee lifecycle metadata across 10 dimensions:

- 🏢 Departments and Job Roles
- 👥 Employee Demographics and Metadata
- 💰 Salary Progression Over Time
- 📈 Performance Reviews and Bonus Eligibility
- 🎓 Promotion Events
- 🚪 Attrition Events and Exit Insights
- 🔍 Hiring Sources and Referral Information

🔧 Key Engineering Highlights:
- Passwords securely retrieved via environment variables
- Datetime formatting applied consistently before database insertion
- Efficient chunk-based loading for large datasets
- Clear table separation aligned with analytical and relational schemas

🎯 Use Cases:
- Power BI / Tableau dashboards for HR metrics
- Machine learning pipelines (attrition prediction, salary modeling)
- Full-stack app development with realistic sample data
- SQL practice and schema optimization for career growth

This notebook is designed for modularity, security, and clarity — ideal for personal portfolios, employer showcase, or client delivery. Combine this with the data generator notebook for a complete synthetic HR data pipeline 🔁.

---